### AUTH

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize(project='ee-USERNAME/PROJECT')

### **Geometries**

In [ ]:
import ee
import folium
from IPython.display import HTML

#Initialize the Earth Engine API
ee.Initialize()

#Define the study area
geometryPINEGULCH = ee.Geometry.Polygon(
        [[[-108.80164742558028, 39.28402267340665],
          [-108.23722481815841, 39.29358857295844],
          [-108.23997140018966, 39.56831366828538],
          [-108.79752755253341, 39.553491364366145]
          ]])

geometryGRIZZLYCREEK = ee.Geometry.Polygon(
        [[[-107.40066503796388, 39.69746161256133],
          [-107.39860510144044, 39.46619798632684],
          [-106.90834020886231, 39.46778823854969],
          [-106.96670507702638, 39.74763415825682],
          [-107.40272497448731, 39.73337830588799]
          ]])

geometryCHERRYCANYON = ee.Geometry.Polygon(
        [[[-103.53011965523325, 37.51499254964799],
          [-103.53011965523325, 37.336135915833296],
          [-103.33785891304575, 37.33176822282621],
          [-103.33785891304575, 37.519349567331446]]]);

geometrySILVERCREEK = ee.Geometry.Polygon(
        [[[-105.29672060223413, 37.6911297494477],
          [-105.29672060223413, 37.35348518722596],
          [-104.96850404949976, 37.35348518722596],
          [-104.96850404949976, 37.6911297494477]]]);
# 2017-07-19

geometryDECKER = ee.Geometry.Polygon(
        [[[-106.02963654395263, 38.479133428395656],
          [-106.02963654395263, 38.38392784402862],
          [-105.91977326270263, 38.38392784402862],
          [-105.91977326270263, 38.479133428395656]]]);

# 2019-09-08

geometryLAKECHRIST = ee.Geometry.Polygon(
        [[[-107.09745152809535, 39.473198422531205],
          [-107.09745152809535, 39.36445604809376],
          [-106.97797520973597, 39.36445604809376],
          [-106.97797520973597, 39.473198422531205]]]);
#2018-07-30

geometryPALATEAU = ee.Geometry.Polygon(
        [[[-108.55952873183622, 37.67620981411429],
          [-108.55952873183622, 37.534774582410634],
          [-108.39198722792997, 37.534774582410634],
          [-108.39198722792997, 37.67620981411429]]]);

geometryHI = ee.Geometry.Polygon(
        [[[-156.7579797108181, 21.083739928456865],
          [-156.7579797108181, 20.707830399739446],
          [-156.17982419323997, 20.707830399739446],
          [-156.17982419323997, 21.083739928456865]]]);


geometries_dict = {
    'PineGulch': geometryPINEGULCH,
    'GrizzlyCreek': geometryGRIZZLYCREEK,
    'CherryCanyon': geometryCHERRYCANYON,
    'SilverCreek': geometrySILVERCREEK,
    'Decker': geometryDECKER,
    'LakeChrist': geometryLAKECHRIST,
    'Palateau': geometryPALATEAU,
    'HI': geometryHI
}

geometries = [
    geometryPINEGULCH,
    geometryGRIZZLYCREEK,
    geometryCHERRYCANYON,
    geometrySILVERCREEK,
    geometryDECKER,
    geometryLAKECHRIST,
    geometryPALATEAU,
    geometryHI
]

geometryHI.getInfo()

In [ ]:
def get_fire_boundary(event_id):
    dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')
    filtered_dataset = dataset.filter(ee.Filter.eq('Event_ID', event_id))

    if filtered_dataset.size().getInfo() > 0:
        selected_fire = ee.Feature(filtered_dataset.first())
        fire_geometry = selected_fire.geometry()
        geometry_type = fire_geometry.type().getInfo()

        if geometry_type == 'Polygon':
            coordinates = fire_geometry.coordinates().getInfo()
            fire_polygon = ee.Geometry.Polygon(coordinates)
            return fire_polygon
        elif geometry_type == 'GeometryCollection':
            geometries = fire_geometry.geometries().getInfo()
            fire_multipolygon = ee.Geometry.MultiPolygon(geometries)
            return fire_multipolygon
        else:
            print(f"Unsupported geometry type: {geometry_type}")
            return None
    else:
        print(f"No features found with the specified Event_ID: {event_id}")
        return None


event_id_PineGulch = 'CO3933610852620200731'
event_id_GrizzlyCreek = 'CO3957210726620200810'
event_id_CherryCanyon = 'CO3736710345020200520'
event_id_Decker = 'CO3840910600420190908'
event_id_LakeChrist = 'CO3937110704320180703'
event_id_Palateau = 'CO3765810847420180722'
event_id_SilverCreek = 'CO4022310665520180719'


event_id_dict = {
    'PineGulch': 'CO3933610852620200731',
    'GrizzlyCreek': 'CO3957210726620200810',
    'CherryCanyon': 'CO3736710345020200520',
    'Decker': 'CO3840910600420190908',
    'LakeChrist': 'CO3937110704320180703',
    'Palateau': 'CO3765810847420180722',
    'SilverCreek': 'CO4022310665520180719'
}

fire_boundaries = {}

#for fire_name, event_id in event_id_dict.items():
#    fire_boundary = get_fire_boundary(event_id)
#
#    if fire_boundary is not None:
#        fire_boundaries[fire_name] = fire_boundary
#        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
#        print(fire_boundaries[fire_name].getInfo())

for fire_name, polygon in geometries_dict.items():
    #fire_boundary = get_fire_boundary(polygon)

    if polygon is not None:
        fire_boundaries[fire_name] = polygon
        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
        print(fire_boundaries[fire_name].getInfo())

### **DATES**

In [ ]:

fire_dates = {
    'PineGulch': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-06-18',
        'postfire_start': '2020-09-20',
        'postfire_end': '2020-10-28'
    },
    'GrizzlyCreek': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-07-18',
        'postfire_start': '2020-09-06',
        'postfire_end': '2020-11-01'
    },
    'CherryCanyon':{
        'prefire_start': '2020-04-20',
        'prefire_end': '2020-05-15',
        'postfire_start': '2020-07-01',
        'postfire_end': '2020-08-15'
    },
    'SilverCreek': { #Done
        'prefire_start': '2021-06-01',
        'prefire_end': '2021-08-10',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-30'
    },
    'Decker': {
        'prefire_start': '2019-05-30',
        'prefire_end': '2019-07-25',
       'postfire_start': '2020-5-10',
       'postfire_end': '2020-6-20'
    },
    'LakeChrist': { # done
        'prefire_start': '2017-08-01',
        'prefire_end': '2017-10-15',
        'postfire_start': '2018-09-15',
        'postfire_end': '2018-10-30'
    },
    'Palateau': { #Done
        'prefire_start': '2018-05-10',
        'prefire_end': '2018-06-20',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-20'
    },
    'HI': { #Done
        'prefire_start': '2022-08-10',
        'prefire_end': '2022-09-20',
        'postfire_start': '2023-08-13',
        'postfire_end': '2023-09-20'
    }
}

fire_names = [
    'PineGulch',
    'GrizzlyCreek',
    'CherryCanyon',
    'Decker',
    'LakeChrist',
    'Palateau',
    'SilverCreek',
    "HI"
]

### **Sentinel 1**


In [ ]:
def create_S1_collection(start_date, end_date, geometry):
    print('Start date: ', start_date)
    print('End date: ', end_date)
    collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) # Add this line
    #print('Collection size: ', collection.size().getInfo())

    return collection

def calculate_dnbr_and_classification_s1(prefire_start, prefire_end, postfire_start, postfire_end, geometry):
    #geometry = geometryCHERRYCANYON

    prefire = create_S1_collection(prefire_start, prefire_end, geometry)
    postfire = create_S1_collection(postfire_start, postfire_end, geometry)

    print('prefire size: ', prefire.size().getInfo())
    print('postfire size: ', postfire.size().getInfo())

    vh_prefire = prefire.select(['VH']).median().clip(geometry)
    vh_postfire = postfire.select(['VH']).median().clip(geometry)

    dNBR_S1 = vh_prefire.divide(vh_postfire).multiply(1.0)

    thresholds = ee.Image([-1, -0.025, -0.0125, -0.005, 0.005, 0.0135, 0.027, 0.045, 0.1, 1])
    classified_S1 = dNBR_S1.lt(thresholds).reduce('sum').toInt()

    return dNBR_S1, classified_S1, vh_prefire, vh_postfire

In [ ]:
dnbr_s1_layers = {}
classification_s1_layers = {}
dnbr_and_classification_s1_results = {}

#for fire_name, fire_boundary in fire_boundaries.items():
#    dates = fire_dates[fire_name]
#    prefire_start = dates["prefire_start"]
#    prefire_end = dates["prefire_end"]
#    postfire_start = dates["postfire_start"]
#    postfire_end = dates["postfire_end"]
#
#    dnbr_s1, classification_s1 = calculate_dnbr_and_classification_s1(
#        prefire_start, prefire_end, postfire_start, postfire_end, fire_boundary
#    )
#
#    dnbr_and_classification_s1_results[fire_name] = {"dnbr_s1": dnbr_s1, "classification_s1": classification_s1}

dnbr_and_classification_s1_results = {}
vh_prefire_results = {}
vh_postfire_results = {}

for fire_name, polygon in geometries_dict.items():

    print('Fire name: ', fire_name)
    print('Polygon coordinates: ', polygon.getInfo())

    dates = fire_dates[fire_name]
    prefire_start = dates["prefire_start"]
    prefire_end = dates["prefire_end"]
    postfire_start = dates["postfire_start"]
    postfire_end = dates["postfire_end"]

    dnbr_s1, classification_s1, vh_prefire, vh_postfire = calculate_dnbr_and_classification_s1(
        prefire_start, prefire_end, postfire_start, postfire_end, polygon
    )

    dnbr_and_classification_s1_results[fire_name] = {"dnbr_s1": dnbr_s1, "classification_s1": classification_s1}
    vh_prefire_results[fire_name] = vh_prefire
    vh_postfire_results[fire_name] = vh_postfire
    print(dnbr_and_classification_s1_results[fire_name]["dnbr_s1"].getInfo())


In [ ]:
sld_intervals_S1_sub = ('<RasterSymbolizer>' +
'<ColorMap type="intervals" extended="false" >' +
#'<ColorMapEntry color="#a41fd6" quantity="-1" label="-1" />' +
'<ColorMapEntry color="#7a8737" quantity="-0.025" label="-0.025"/>' +
'<ColorMapEntry color="#7a8737" quantity="-0.0125" label="-0.0125" />' +
'<ColorMapEntry color="#acbe4d" quantity="-0.005" label="-0.005" />' +
'<ColorMapEntry color="#0ae042" quantity="0.005" label="0.005" />' +
'<ColorMapEntry color="#fff70b" quantity="0.0135" label="0.0135" />' +
'<ColorMapEntry color="#ffaf38" quantity="0.027" label="0.027" />' +
'<ColorMapEntry color="#ff641b" quantity="0.045" label="0.045" />' +
'<ColorMapEntry color="#a41fd6" quantity="0.1" label="0.1" />' +
'<ColorMapEntry color="#a41fd6" quantity="1" label="1" />' +
'</ColorMap>' +
'</RasterSymbolizer>')

sld_intervals_S1_ratio = ('<RasterSymbolizer>' +
'<ColorMap type="intervals" extended="false" >' +
'<ColorMapEntry color="#7a8737" quantity="0.85" label="0.85" />' +
'<ColorMapEntry color="#acbe4d" quantity="0.87" label="0.87" />' +
'<ColorMapEntry color="#0ae042" quantity="0.9" label="0.90" />' +
'<ColorMapEntry color="#fff70b" quantity="0.95" label="0.95" />' +
'<ColorMapEntry color="#ffaf38" quantity="1.0" label="1.0" />' +
'<ColorMapEntry color="#ff641b" quantity="1.25" label="1.25" />' +
'<ColorMapEntry color="#a41fd6" quantity="2.5" label="2.0" />' +
'<ColorMapEntry color="#a41fd6" quantity="4.5" label="4.5" />' +
'</ColorMap>' +
'</RasterSymbolizer>')


def create_S1_collection(start_date, end_date, geometry):
    print('Start date: ', start_date)
    print('End date: ', end_date)
    collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq('instrumentMode', 'IW'))\
        .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
        .sort('system:time_start') # Add this line
    #print('Collection size: ', collection.size().getInfo())
    #.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\

    return collection



def create_MODIS_collection(start_date, end_date, geometry):
    print('Start date: ', start_date)
    print('End date: ', end_date)
    collection = ee.ImageCollection('MODIS/061/MOD14A1') \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    return collection


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#fire_names = [
#    'PineGulch',
#    'GrizzlyCreek',
#   'CherryCanyon',
#    'Decker',
#    'LakeChrist',
#    'Palateau',
#    'SilverCreek'
#]


fire_name = fire_names[6]

poly =  geometries_dict[fire_name]
dates = fire_dates[fire_name]
prefire_start = dates["prefire_start"]
prefire_end = dates["prefire_end"]
postfire_start = dates["postfire_start"]
postfire_end = dates["postfire_end"]

PG_image1_pre = create_S1_collection(prefire_start, prefire_end, poly)
PG_image1_post = create_S1_collection(postfire_start, postfire_end, poly)
#PG_image1_post = PG_image1_post1.map(exponential_scaler)


import time
acq_times_pre = PG_image1_pre.aggregate_array('system:time_start').getInfo()
acq_times_post = PG_image1_post.aggregate_array('system:time_start').getInfo()

time_list_pre = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times_pre]
time_list_post = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times_post]

print(time_list_pre)
print(time_list_post)

im_list_pre = PG_image1_pre.toList(PG_image1_pre.size())
im_list_post = PG_image1_post.toList(PG_image1_post.size())
list_length_post = im_list_post.size().getInfo()

im_pre = ee.Image(PG_image1_pre.median()).select('VH').clip(poly)
#im_post = ee.Image(im_list_post.get(list_length_post - 3)).select('VH').clip(poly)
im_post = ee.Image(PG_image1_post.median()).select('VH').clip(poly)

ratio = im_pre.divide(im_post)
sub = im_pre.subtract(im_post)
normal_diff = (im_pre.subtract(im_post)).divide((im_pre.add(im_post)))


#######################################
#filtered_pre = im_pre.focal_median(radius=2, kernelType='square', iterations=1)
#filtered_post = im_post.focal_median(radius=2, kernelType='square', iterations=1)

##ratio_filtered = filtered_pre.divide(filtered_post)
#sub_filtered = filtered_pre.subtract(filtered_post)
#######################################

thresholds = ee.Image([-1, -0.025, -0.0125, -0.005, 0.005, 0.0135, 0.027, 0.045, 0.1, 1])
custom_palette = ['#7a8737', '#7a8737', '#acbe4d', '#0ae042', '#fff70b', '#ffaf38', '#ff641b', '#a41fd6', '#a41fd6']

#classified = ratio.lt(thresholds).reduce('sum').toInt()

MODIS_image_post = create_MODIS_collection(postfire_start, postfire_end, poly)
fireMaskVis = {
  'min': 0.0,
  'max': 6000.0,
  'bands': ['MaxFRP', 'FireMask', 'FireMask'],
}

smooth_palette = ['black', 'gray', 'white', 'pink', 'red']

mp = folium.Map(location= [39.40, -108.5], zoom_start=8)

mp.add_ee_layer(im_pre,
                {'min': -0.05, 'max': 0.05, 'palette': ['black','white']}, 'pre')

mp.add_ee_layer(im_post,
                {'min': -0.05, 'max': 0.05, 'palette': ['black','white']}, 'post')
#################################
#mp.add_ee_layer(filtered_pre,
#                {'min': -0.05, 'max': 0.05, 'palette': ['black','white']}, 'pre-filtered')

#mp.add_ee_layer(filtered_post,
#                {'min': -0.05, 'max': 0.05, 'palette': ['black','white']}, 'post-filtered')
#################################
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 5, 'palette': ['white','black']}, 'Ratio')
mp.add_ee_layer(sub,
                {'min': 0, 'max': 0.015, 'palette': ['black', 'white']}, 'sub')

mp.add_ee_layer(normal_diff,
                {'min': 1, 'max': 1, 'palette':['white','black']}, 'normal Diff')

#mp.add_ee_layer(classified, {'min': -1, 'max': 1, 'palette': custom_palette}, 'Classification')
folium.TileLayer(tiles=sub.sldStyle(sld_intervals_S1_sub).getMapId()['tile_fetcher'].url_format, attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>', overlay=True, name='dNBR classified sub').add_to(mp)
folium.TileLayer(tiles=ratio.sldStyle(sld_intervals_S1_ratio).getMapId()['tile_fetcher'].url_format, attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>', overlay=True, name='dNBR classified ratio').add_to(mp)
#mp.add_ee_layer(ratio_filtered,
#                {'min': .85, 'max': 4.5, 'palette': ['white', 'black']}, 'Ratio-filtered')

#mp.add_ee_layer(sub_filtered,
#                {'min': 0, 'max': 0.015, 'palette': ['white', 'black']}, 'sub-filtered')

#['blue', 'white', 'red'] as the palette, where blue represents decrease, white represents no change, and red represents increase.

#mp.add_ee_layer(classified, {'min': -0.05, 'max': 0.05, 'palette': ['black', 'white']}, 'Classification')
mp.add_ee_layer(MODIS_image_post.median(), fireMaskVis, 'MODIS postfire')

#mp.add_child(folium.LayerControl())
folium.LayerControl().add_to(mp)

display(mp)

In [ ]:
list_length_post = im_list_post.size().getInfo()
mp2 = folium.Map(location= [39.40, -108.5], zoom_start=1)


for i in range(list_length_post):
    #image from the collection
    im_post = ee.Image(im_list_post.get(i)).select('VH').clip(poly)


    mp2.add_ee_layer(im_post,
                    {'min': -0.05, 'max': 0.05, 'palette': ['black', 'white']},
                    'Post fire image {}'.format(i+1))


mp2.add_child(folium.LayerControl())

display(mp2)

### **EXPORT S1** only run when wanting to export S1 images from above. Not needed Every time.

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

google_drive_path_S1 = "/content/drive/MyDrive/Cleaned_Fire_images_TIFF/"
os.chdir("/content/drive/MyDrive/Cleaned_Fire_images_TIFF/")

scale = 10

#export def
def initiate_export(image, description, viz_params, region, scale=10):
    image = image.visualize(**viz_params)
    output_path = f'{description}.tif'
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder='Cleaned_Fire_images_TIFF',
        fileNamePrefix=output_path[:-4],
        region=region,
        scale=scale,
        crs='EPSG:4326'
    )
    task.start()
    print(f"Export task started for {description}")

#Visualization parameters
viz_params_pre = {'min': -0.05, 'max': 0.05, 'palette': ['black', 'white']}
viz_params_post = {'min': -0.05, 'max': 0.05, 'palette': ['black', 'white']}
viz_params_sub = {'min': 0, 'max': 0.015, 'palette': ['white', 'black']}
viz_params_ratio = {'min': 0, 'max': 5, 'palette': ['white', 'black']}

#Initialize
#initiate_export(im_pre, f'S1_pre_{fire_name}_box', viz_params_pre, fire_boundaries[fire_name])
#initiate_export(im_post, f'S1_post_{fire_name}_box', viz_params_post, fire_boundaries[fire_name])
#initiate_export(sub, f'S1_sub_{fire_name}_box', viz_params_sub, fire_boundaries[fire_name])
initiate_export(ratio, f'S1_ratio_{fire_name}_box_ratio', viz_params_ratio, fire_boundaries[fire_name])